## Validate Incremental Load

Let us analyze GHActivity Data that is added to the target by our Spark Streaming Process.
* Location: **/user/{username}/itv-github/streaming/bronze/data/ghactivity**.
* As the files are in parquet format, we can use `spark.read.format('parquet')` to read these files.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Incremental Loads using Spark Structurd Streaming'). \
    master('yarn'). \
    getOrCreate()

In [ ]:
!hdfs dfs -ls /user/${USER}/itv-github/streaming/bronze/data/ghactivity/created_year=2021/created_month=01

In [ ]:
ghactivity = spark. \
    read. \
    parquet(f'/user/{username}/itv-github/streaming/bronze/data/ghactivity')

In [ ]:
ghactivity.count()

In [ ]:
ghactivity.printSchema()

In [ ]:
ghactivity. \
  filter("type = 'CreateEvent' AND payload.ref_type = 'repository'"). \
  count()

* Get count by date to confirm the counts by date.

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
ghactivity. \
  groupby(to_date('created_at')). \
  count(). \
  show()

In [ ]:
ghactivity. \
  groupby('year', 'month', 'dayofmonth'). \
  count(). \
  show()

In [ ]:
ghactivity. \
  filter("type = 'CreateEvent' AND payload.ref_type = 'repository'"). \
  groupby(to_date('created_at')). \
  count(). \
  show()